# LOFAR ionospheric scintillation data tutorial part 1
Maaijke Mevius, June 2023

We will inspect beamformed data of single stations pointing at one of the brightest radio sources in the Northern hemisphere.

The datasets we will use are [Dynspec_rebinned_L262603_SAP000.h5](https://filesender.surf.nl/download.php?token=da8a7b03-ad30-4e94-80cf-9255181240f6&files_ids=14358098), [LOFAR_20150302_170000_CS001LBA_LBA_OUTER_Cas-A.fits](https://spaceweather.astron.nl/SolarKSP/data/atdb_process/scintillation_preview/3155/262603/fits_files/LOFAR_20150302_170000_CS001LBA_LBA_OUTER_Cas-A.fits) and [LOFAR_20230503_100000_CS032LBA_LBA_OUTER_.fits](https://spaceweather.astron.nl/SolarKSP/data/atdb_process/scintillation_preview/4603/888136/fits_files/LOFAR_20230503_100000_CS032LBA_LBA_OUTER_.fits). Please download these to the `data/` subdirectory of the working directory of this notebook.

First some basic imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

The data format we will work with is either hdf5 or fits. Fits files can easily be read using the `astropy.io.fits` package. For the hdf5 data we use `h5py`:

In [ ]:
import h5py
from astropy.io import fits

The raw LOFAR beamformed data is generally stored in hdf5 format. Depending on how much averaging has happened before storing the data, these files can become very large. Let's start by inspecting one of the hdf5 files.  

In [ ]:
myfile = h5py.File("data/Dynspec_rebinned_L262603_SAP000.h5")

An hdf5 file contains data arrays and metadata in a tree-like structure. `myfile` is pointing at the root of the file. You can access the branches below the root by checking the keys:

In [ ]:
root_keys = myfile.keys()
print(root_keys)

To plot the keys of any of the dynamic spectra:

In [ ]:
for key in root_keys:
    print(myfile[key].keys())
    break

The metadata can be accessed similarly via the `.attrs` attribute.

In [ ]:
print(list(myfile.attrs))

### Excercise

Try to familiarize yourself with the data format. What is the time resolution of the data? How many stations do we have data for? Can you get the frequency and time range of the data?

In [ ]:
for mykey in myfile.attrs:
    print(mykey, myfile.attrs[mykey], sep="\t")

In [ ]:
from astropy.time import Time

In [ ]:
freq_start = myfile.attrs["OBSERVATION_FREQUENCY_MIN"]
freq_end = myfile.attrs["OBSERVATION_FREQUENCY_MAX"]
time_start = Time(myfile.attrs["OBSERVATION_START_UTC"][:22])
time_end = Time(myfile.attrs["OBSERVATION_END_UTC"][:22])

In [ ]:
for mykey in myfile["DYNSPEC_000"].attrs:
    print(mykey, myfile["DYNSPEC_000"].attrs[mykey], sep="\t")

### Excercise

The time frequency data can be accessed via the individual `DYNSPEC_###`. Each branch contains the data of a separate station. Find the shape of the data of the first station.

In [ ]:
data = myfile["DYNSPEC_000"]["DATA"]

In [ ]:
print(data.shape)

The data size is 57209 timeslot x 350 frequency channels. Note that `data` is only a pointer to the data, it has not been read into memory yet. Although you might be able to read all of it into the memory of your computer, this is in general not the case, and you might want to read the data in parts before processing. You can treat the data pointer as if it was a numpy array. Let's try to make some images.

### Excercise

Make a time-frequency spectrum of every 10th time sample. **Hint:** Use `plt.imshow`, add axis labels by using the `extent` keyword. The frequency range can be found in the attrs. You can slice the data in numpy by using this syntax: `data[start:end:stepsize]`.

In [ ]:
from matplotlib.dates import ConciseDateFormatter, date2num

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    data[::10, :, 0].T,
    origin="lower",
    interpolation="none",
    aspect="auto",
    extent=[
        date2num(time_start.datetime),
        date2num(time_end.datetime),
        freq_start,
        freq_end,
    ],
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

Apart from the stripes at the bottom (what are those? )Not much to see in this one. Try using `np.log10()` on the data, to get a better dynamic range.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    np.log10(data[::10, :, 0].T),
    origin="lower",
    interpolation="none",
    aspect="auto",
    extent=[
        date2num(time_start.datetime),
        date2num(time_end.datetime),
        freq_start,
        freq_end,
    ],
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

### Excercise: 

The bandpass is the frequency response of the telescope. Make a plot of the bandpass, use a logarithmic scale for the y-axis. **Hint:** use numpy `median` function on the time axis.

In [ ]:
bandpass = np.median(data[::10, :, 0], axis=0)
freqs = np.linspace(freq_start, freq_end, bandpass.shape[0])

In [ ]:
fig, ax = plt.subplots()
ax.plot(freqs, bandpass)
ax.set_yscale("log")
ax.set_xlabel("Frequency (MHz)")
ax.set_ylabel("Median intensity");

### 
Apart from the strong RFI below 20 MHz, we can nicely recognize the LOFAR LBA dipole response, which has a resonance frequency around 58 MHz. We can now normalize the power spectrum by dividing or subtracting the bandpass.

### Excercise:

Make the time frequency power spectrum again but now normalized with the bandpass. Since the data (apart from the RFI) should now be close to 1 you can use the `vmin` and `vmax` parameters to select the interesting vertical scale (e.g. the 10% and 90% percentile of the data values).

Structures start to emerge. What do you see?  

In [ ]:
normalized_data = data[::10, :, 0].T / bandpass[:, np.newaxis]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    normalized_data,
    origin="lower",
    interpolation="none",
    aspect="auto",
    extent=[
        date2num(time_start.datetime),
        date2num(time_end.datetime),
        freq_start,
        freq_end,
    ],
    vmin=0.9,
    vmax=1.1,
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

### Excercise:

Now make the same plot of a couple of stations. Do you notice something?

### 
The scintillation index (S4) is defined as the normalised intensity variation: 
$S4^2 = {{<I^2> - <I>^2}\over{<I>^2}} = \frac{\text{variance}(I)}{\text{mean}(I)^2}$
Typically it is calculated over a window of 60s.
S4 gives a single number measure of the ionospheric conditions. 

### Excercise:

Calculate S4 over a window of 60s (~600 timeslots on the raw data) for all times. Either select a single frequency (without RFI) or use the median over a range of frequencies. Plot the result.

In [ ]:
data_no_rfi = data[:, 200:230 ,0] / bandpass[np.newaxis,200:230]

In [ ]:
median_data = np.median(data_no_rfi, axis=1)

In [ ]:
ntimes = median_data.shape[0]

In [ ]:
# Make sure the data can be divided evenly into chunks of 600 timeslots
rounded_ntimes = ntimes - ntimes % 600
median_data = median_data[0:rounded_ntimes]

In [ ]:
# Divide the data into time chunks and take the variance of each
median_data_chunks = np.reshape(median_data, (-1, 600))
median_data_chunks.shape

In [ ]:
s4 = np.sqrt(
    (np.mean(median_data_chunks**2, axis=1) - np.mean(median_data_chunks, axis=1) ** 2)
     / (np.mean(median_data_chunks, axis=1) ** 2))

In [ ]:
s4_times = np.linspace(time_start.mjd, time_end.mjd, len(s4))
fig, ax = plt.subplots()
ax.plot(Time(s4_times, format='mjd').datetime, s4);
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_ylim(None, 0.25)

In [ ]:
plt.plot(median_data)